In [1]:
# your code here
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import os

In [2]:
# https://connect.deezer.com/oauth/auth.php?app_id=YOUR_APP_ID&redirect_uri=YOUR_REDIRECT_URI&perms=basic_access,email 
#https://api.deezer.com/version/service/id/method/?parameters
#The number of requests per second is limited to 50 requests / 5 seconds.
#Pagination : https://api.deezer.com/playlist/4341978/tracks?index=0&limit=10 OR https://api.deezer.com/playlist/4341978/tracks?index=3&limit=7


In [3]:
#url = 'https://api.deezer.com/chart?index=0&limit=10'
h={'user-agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:68.0) Gecko/20100101 Firefox/68.0'}

In [4]:
""" VERSION NICO

pages_df = []

for i in range(1):
    k=50*i
    url=f'https://api.deezer.com/chart?index={k}&limit=50'
    deezer = requests.get(url,headers=h)
    dzr_results = deezer.json()

    extract = lambda col :json_normalize(dzr_results, record_path=[col, 'data'], meta=[[col, 'total']])

    cols = ['albums', 'artists', 'playlists', 'podcasts', 'tracks']

    dfs = [extract(col) for col in cols]

    data2 = dfs[0]
    for df in dfs[1:]:
        data2 = data2.merge(df, data2, on='tracks.total')
    
    pages_df.append(data2)
    
pd.concat(pages_df, ...)"""

" VERSION NICO\n\npages_df = []\n\nfor i in range(1):\n    k=50*i\n    url=f'https://api.deezer.com/chart?index={k}&limit=50'\n    deezer = requests.get(url,headers=h)\n    dzr_results = deezer.json()\n\n    extract = lambda col :json_normalize(dzr_results, record_path=[col, 'data'], meta=[[col, 'total']])\n\n    cols = ['albums', 'artists', 'playlists', 'podcasts', 'tracks']\n\n    dfs = [extract(col) for col in cols]\n\n    data2 = dfs[0]\n    for df in dfs[1:]:\n        data2 = data2.merge(df, data2, on='tracks.total')\n    \n    pages_df.append(data2)\n    \npd.concat(pages_df, ...)"

In [5]:
""" VERSION SANDRINE
df_deezer =pd.DataFrame()
for i in range(5):
    k=50*i
    url=f'https://api.deezer.com/chart?index={k}&limit=50'
    deezer = requests.get(url,headers=h)
    dzr_results = deezer.json()
    flattened_data = json_normalize(dzr_results)
    
    #flattened_data1 = json_normalize(flattened_data)
    df_deezer = df_deezer.append(flattened_data,sort=True)

df_deezer.to_json('deezer.json', orient='records')
df_deezer"""

" VERSION SANDRINE\ndf_deezer =pd.DataFrame()\nfor i in range(5):\n    k=50*i\n    url=f'https://api.deezer.com/chart?index={k}&limit=50'\n    deezer = requests.get(url,headers=h)\n    dzr_results = deezer.json()\n    flattened_data = json_normalize(dzr_results)\n    \n    #flattened_data1 = json_normalize(flattened_data)\n    df_deezer = df_deezer.append(flattened_data,sort=True)\n\ndf_deezer.to_json('deezer.json', orient='records')\ndf_deezer"

In [6]:
pages_df =[]

for i in range(3):
    j=1
    k=50*i
    url=f'https://api.deezer.com/chart?index={k}&limit=50'
    deezer = requests.get(url,headers=h)
    dzr_results = deezer.json()
    cols = ['albums', 'artists', 'playlists', 'podcasts', 'tracks']
    for col in cols : 
        dfs = [json_normalize(data=dzr_results, record_path=[col, 'data'], meta=[['albums', 'total'], ['artists', 'total'],['playlists', 'total'], ['tracks', 'total']])]
    data2 = dfs[0]
    for df in dfs[1:]:
        data2 = data2.merge(df, data2, on=['albums', 'total'], how='left')
    pages_df.append(data2)

In [7]:
dzz = pd.concat(pages_df, ignore_index=True)
dzz.to_json('deezer.json', orient='records')
dzz.head(3)

,album,artist,duration,explicit_content_cover,explicit_content_lyrics,explicit_lyrics,id,link,position,preview,rank,title,title_short,title_version,type,albums.total,artists.total,playlists.total,tracks.total
0,"{'id': 104455482, 'title': 'Carte Blanche', 'c...","{'id': 482758, 'name': 'DJ Snake', 'link': 'ht...",185,0,0,False,716383902,https://www.deezer.com/track/716383902,1,https://cdns-preview-4.dzcdn.net/stream/c-4783...,999826,Loco Contigo,Loco Contigo,,track,50,50,50,50
1,"{'id': 98518002, 'title': 'Elle est bonne sa m...","{'id': 9134324, 'name': 'Vegedream', 'link': '...",197,0,0,True,687546992,https://www.deezer.com/track/687546992,2,https://cdns-preview-3.dzcdn.net/stream/c-39bc...,998080,Elle est bonne sa mère,Elle est bonne sa mère,,track,50,50,50,50
2,"{'id': 100856872, 'title': 'Señorita', 'cover'...","{'id': 5962948, 'name': 'Shawn Mendes', 'link'...",190,0,0,False,698905582,https://www.deezer.com/track/698905582,3,https://cdns-preview-5.dzcdn.net/stream/c-58c2...,998970,Señorita,Señorita,,track,50,50,50,50


In [8]:
data = pd.read_json('deezer.json', orient='records')
data.index.name='index'
data.artist[0]

{'id': 482758,
 'name': 'DJ Snake',
 'link': 'https://www.deezer.com/artist/482758',
 'picture': 'https://api.deezer.com/artist/482758/image',
 'picture_small': 'https://cdns-images.dzcdn.net/images/artist/02b4b508aa974d5f6e8348e2186dd49f/56x56-000000-80-0-0.jpg',
 'picture_medium': 'https://cdns-images.dzcdn.net/images/artist/02b4b508aa974d5f6e8348e2186dd49f/250x250-000000-80-0-0.jpg',
 'picture_big': 'https://cdns-images.dzcdn.net/images/artist/02b4b508aa974d5f6e8348e2186dd49f/500x500-000000-80-0-0.jpg',
 'picture_xl': 'https://cdns-images.dzcdn.net/images/artist/02b4b508aa974d5f6e8348e2186dd49f/1000x1000-000000-80-0-0.jpg',
 'radio': True,
 'tracklist': 'https://api.deezer.com/artist/482758/top?limit=50',
 'type': 'artist'}

In [9]:
extract = json_normalize(data['album'])
extract2 = json_normalize(data['artist'])
data = pd.concat([extract,data], join="inner", axis=1)

In [10]:
data = pd.concat([extract2,data], join="inner", axis=1)
data.head()

,id,link,name,picture,picture_big,picture_medium,picture_small,picture_xl,radio,tracklist,...,link,playlists.total,position,preview,rank,title,title_short,title_version,tracks.total,type
0,482758,https://www.deezer.com/artist/482758,DJ Snake,https://api.deezer.com/artist/482758/image,https://cdns-images.dzcdn.net/images/artist/02...,https://cdns-images.dzcdn.net/images/artist/02...,https://cdns-images.dzcdn.net/images/artist/02...,https://cdns-images.dzcdn.net/images/artist/02...,True,https://api.deezer.com/artist/482758/top?limit=50,...,https://www.deezer.com/track/716383902,50,1,https://cdns-preview-4.dzcdn.net/stream/c-4783...,999826,Loco Contigo,Loco Contigo,,50,track
1,9134324,https://www.deezer.com/artist/9134324,Vegedream,https://api.deezer.com/artist/9134324/image,https://cdns-images.dzcdn.net/images/artist/46...,https://cdns-images.dzcdn.net/images/artist/46...,https://cdns-images.dzcdn.net/images/artist/46...,https://cdns-images.dzcdn.net/images/artist/46...,True,https://api.deezer.com/artist/9134324/top?limi...,...,https://www.deezer.com/track/687546992,50,2,https://cdns-preview-3.dzcdn.net/stream/c-39bc...,998080,Elle est bonne sa mère,Elle est bonne sa mère,,50,track
2,5962948,https://www.deezer.com/artist/5962948,Shawn Mendes,https://api.deezer.com/artist/5962948/image,https://cdns-images.dzcdn.net/images/artist/69...,https://cdns-images.dzcdn.net/images/artist/69...,https://cdns-images.dzcdn.net/images/artist/69...,https://cdns-images.dzcdn.net/images/artist/69...,True,https://api.deezer.com/artist/5962948/top?limi...,...,https://www.deezer.com/track/698905582,50,3,https://cdns-preview-5.dzcdn.net/stream/c-58c2...,998970,Señorita,Señorita,,50,track
3,15166511,https://www.deezer.com/artist/15166511,Lil Nas X,https://api.deezer.com/artist/15166511/image,https://cdns-images.dzcdn.net/images/artist/e6...,https://cdns-images.dzcdn.net/images/artist/e6...,https://cdns-images.dzcdn.net/images/artist/e6...,https://cdns-images.dzcdn.net/images/artist/e6...,True,https://api.deezer.com/artist/15166511/top?lim...,...,https://www.deezer.com/track/699056192,50,4,https://cdns-preview-7.dzcdn.net/stream/c-7bbb...,998420,Old Town Road (Remix),Old Town Road,(Remix),50,track
4,8909272,https://www.deezer.com/artist/8909272,Aya Nakamura,https://api.deezer.com/artist/8909272/image,https://cdns-images.dzcdn.net/images/artist/be...,https://cdns-images.dzcdn.net/images/artist/be...,https://cdns-images.dzcdn.net/images/artist/be...,https://cdns-images.dzcdn.net/images/artist/be...,True,https://api.deezer.com/artist/8909272/top?limi...,...,https://www.deezer.com/track/576851252,50,5,https://cdns-preview-a.dzcdn.net/stream/c-ab16...,983184,Pookie,Pookie,,50,track


In [11]:
# A DROPER data['album']
# A DROPER data['artist']
# Adroper le scolonnes avec Total

data.drop(columns=['album','artist','albums.total','tracks.total','playlists.total','artists.total'], axis=1)

,id,link,name,picture,picture_big,picture_medium,picture_small,picture_xl,radio,tracklist,...,explicit_lyrics,id,link,position,preview,rank,title,title_short,title_version,type
0,482758,https://www.deezer.com/artist/482758,DJ Snake,https://api.deezer.com/artist/482758/image,https://cdns-images.dzcdn.net/images/artist/02...,https://cdns-images.dzcdn.net/images/artist/02...,https://cdns-images.dzcdn.net/images/artist/02...,https://cdns-images.dzcdn.net/images/artist/02...,True,https://api.deezer.com/artist/482758/top?limit=50,...,False,716383902,https://www.deezer.com/track/716383902,1,https://cdns-preview-4.dzcdn.net/stream/c-4783...,999826,Loco Contigo,Loco Contigo,,track
1,9134324,https://www.deezer.com/artist/9134324,Vegedream,https://api.deezer.com/artist/9134324/image,https://cdns-images.dzcdn.net/images/artist/46...,https://cdns-images.dzcdn.net/images/artist/46...,https://cdns-images.dzcdn.net/images/artist/46...,https://cdns-images.dzcdn.net/images/artist/46...,True,https://api.deezer.com/artist/9134324/top?limi...,...,True,687546992,https://www.deezer.com/track/687546992,2,https://cdns-preview-3.dzcdn.net/stream/c-39bc...,998080,Elle est bonne sa mère,Elle est bonne sa mère,,track
2,5962948,https://www.deezer.com/artist/5962948,Shawn Mendes,https://api.deezer.com/artist/5962948/image,https://cdns-images.dzcdn.net/images/artist/69...,https://cdns-images.dzcdn.net/images/artist/69...,https://cdns-images.dzcdn.net/images/artist/69...,https://cdns-images.dzcdn.net/images/artist/69...,True,https://api.deezer.com/artist/5962948/top?limi...,...,False,698905582,https://www.deezer.com/track/698905582,3,https://cdns-preview-5.dzcdn.net/stream/c-58c2...,998970,Señorita,Señorita,,track
3,15166511,https://www.deezer.com/artist/15166511,Lil Nas X,https://api.deezer.com/artist/15166511/image,https://cdns-images.dzcdn.net/images/artist/e6...,https://cdns-images.dzcdn.net/images/artist/e6...,https://cdns-images.dzcdn.net/images/artist/e6...,https://cdns-images.dzcdn.net/images/artist/e6...,True,https://api.deezer.com/artist/15166511/top?lim...,...,False,699056192,https://www.deezer.com/track/699056192,4,https://cdns-preview-7.dzcdn.net/stream/c-7bbb...,998420,Old Town Road (Remix),Old Town Road,(Remix),track
4,8909272,https://www.deezer.com/artist/8909272,Aya Nakamura,https://api.deezer.com/artist/8909272/image,https://cdns-images.dzcdn.net/images/artist/be...,https://cdns-images.dzcdn.net/images/artist/be...,https://cdns-images.dzcdn.net/images/artist/be...,https://cdns-images.dzcdn.net/images/artist/be...,True,https://api.deezer.com/artist/8909272/top?limi...,...,False,576851252,https://www.deezer.com/track/576851252,5,https://cdns-preview-a.dzcdn.net/stream/c-ab16...,983184,Pookie,Pookie,,track
5,5288900,https://www.deezer.com/artist/5288900,Niska,https://api.deezer.com/artist/5288900/image,https://cdns-images.dzcdn.net/images/artist/eb...,https://cdns-images.dzcdn.net/images/artist/eb...,https://cdns-images.dzcdn.net/images/artist/eb...,https://cdns-images.dzcdn.net/images/artist/eb...,True,https://api.deezer.com/artist/5288900/top?limi...,...,True,658757872,https://www.deezer.com/track/658757872,6,https://cdns-preview-8.dzcdn.net/stream/c-84d1...,997582,Médicament,Médicament,,track
6,1479842,https://www.deezer.com/artist/1479842,PLK,https://api.deezer.com/artist/1479842/image,https://cdns-images.dzcdn.net/images/artist/a2...,https://cdns-images.dzcdn.net/images/artist/a2...,https://cdns-images.dzcdn.net/images/artist/a2...,https://cdns-images.dzcdn.net/images/artist/a2...,True,https://api.deezer.com/artist/1479842/top?limi...,...,True,709488522,https://www.deezer.com/track/709488522,7,https://cdns-preview-a.dzcdn.net/stream/c-a1ee...,997103,Problèmes,Problèmes,,track
7,1042268,https://www.deezer.com/artist/1042268,Angèle,https://api.deezer.com/artist/1042268/image,https://cdns-images.dzcdn.net/images/artist/6d...,https://cdns-images.dzcdn.net/images/artist/6d...,https://cdns-images.dzcdn.net/images/artist/6d...,https://cdns-images

In [15]:
data.to_json('deezer_final.json', orient='values')